In [1]:
import dolfinx
#import dxh
import ufl
import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI
from petsc4py import PETSc
from scipy.sparse import csr_matrix
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

Define experiment parameters

In [2]:
final_time = 0.9
number_of_time_slices = 16
number_spatial_mesh_cells = 32
time_step = final_time / number_of_time_slices
include_initial_condition = False
spatial_element_degree = 1
temporal_element_degree = 1
# stabilization parameters
lambda_nitsche = 5 * spatial_element_degree**2
gamma_0 = 1
gamma_1 = 1
gamma_m = 1
#gamma_primal_jump = gamma_primal
gamma_dual = 1
gamma_data = 1e4

Set up temporal basis functions for trial and test function spaces

In [3]:
phi_test = phi_trial = (lambda t: 1-t, lambda t: t)
dt_phi_test = dt_phi_trial = (lambda t: -1, lambda t: 1)

Define integrals in time of temporal basis functions, hardcoded here for simplicity

In [4]:
integral_phi_phi = np.array([[1/3, 1/6], [1/6, 1/3]])
integral_dphi_phi = np.array([[-1/2, 1/2], [-1/2, 1/2]])
integral_dphi_dphi = np.array([[1.0, -1.0], [-1.0, 1.0]])

Define quadrature rule for integrating right-hand side with respect to time

In [5]:
class GaussRadauQuadratureRule:

    def __init__(self, number_of_points):
        self.number_of_points = number_of_points
        gauss_radau = {
            3: (
                [-1, (1 - np.sqrt(6)) / 5, (1 + np.sqrt(6)) / 5],
                [2 / 9, (16 + np.sqrt(6)) / 18, (16 - np.sqrt(6)) / 18]
            ),
            4: (
                [-1, -0.575319, 0.181066, 0.822824],
                [0.125, 0.657689, 0.776387, 0.440924]
            )
        }
        self.points, self.weights = gauss_radau[number_of_points]
        
    def current_pts(self, a, b):
        return [0.5 * (b - a) * pt + 0.5 * (b + a)  for pt in self.points]
    
    def t_weights(self, delta_t):
        return [0.5 * delta_t * w for w in self.weights]

quadrature_rule = GaussRadauQuadratureRule(4) # quad rule for integrating rhs

Construct mesh on one-dimensional unit interval spatial domain

In [6]:
communicator = MPI.COMM_WORLD
#mesh = dolfinx.mesh.create_unit_interval(communicator, number_spatial_mesh_cells)
mesh = dolfinx.mesh.create_unit_square(communicator, number_spatial_mesh_cells,number_spatial_mesh_cells)

Create base vector finite element with continuous Galerkin family and dimension proportional to number slices temporal interval is split into

In [7]:
vector_element = ufl.VectorElement(
    family="CR",
    cell=mesh.ufl_cell(),
    degree=spatial_element_degree,
    dim=2 * number_of_time_slices
)

Create a mixed element corresponding to outer level of nesting for each of four variables (displacement primal, velocity primal, displacement dual, velocity dual) and corresponding function space

In [8]:
mixed_element = ufl.MixedElement([vector_element] * 2)
mixed_function_space = dolfinx.fem.FunctionSpace(mesh, mixed_element)

Construct trial and test functions for primal and dual variables (for both displacement and velocity)

In [9]:
(
    velocity_primal_trial_function,
    velocity_dual_trial_function,
) = ufl.TrialFunctions(mixed_function_space)
(
    velocity_primal_test_function,
    velocity_dual_test_function,
) = ufl.TestFunctions(mixed_function_space)

Create indicator function on spatial domain for where data is available

In [10]:
def data_domain_indicator(spatial_coordinate, lower_bound=0.2, upper_bound=0.8):
    return ((spatial_coordinate[0] <= lower_bound) | (spatial_coordinate[0] >= upper_bound)) * 1

indicator_function_space = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))
data_domain_indicator_function = dolfinx.fem.Function(indicator_function_space)
data_domain_indicator_function.interpolate(data_domain_indicator)

Define function for constructing required measures / integrals on spatial domain and create instances of measures for mesh

In [11]:
def construct_measures(mesh, spatial_element_degree):
    metadata = {"quadrature_degree": 2 * spatial_element_degree + 3}
    cell_integral = ufl.Measure("cell", domain=mesh, metadata=metadata)
    exterior_facet_integral = ufl.Measure("exterior_facet", domain=mesh, metadata=metadata)
    interior_facet_integral = ufl.Measure("interior_facet", domain=mesh, metadata=metadata)
    return cell_integral, exterior_facet_integral, interior_facet_integral

cell_integral, exterior_facet_integral, interior_facet_integral = construct_measures(mesh, spatial_element_degree)
spatial_coordinate = ufl.SpatialCoordinate(mesh)

$A_1(u,w)=\tau \sum^{N}_{n=1}((\partial_{\tau}u^n,w^n)+a_h(u^n,w^n))$

In [12]:
def problem_bilinear_form( 
    #velocity_primal_trial_function,
    velocity_primal_trial_function_current,
    velocity_primal_trial_function_previous,
    velocity_primal_test_function,
    #integral_phi_phi,
    #integral_dphi_phi,
    time_step,
    mesh,
    spatial_element_degree,
):
    cell_integral, exterior_facet_integral, _ = construct_measures(
        mesh, spatial_element_degree
    )
    facet_normal = ufl.FacetNormal(mesh)
    return (
        #integral_dphi_phi
         ufl.inner(
            velocity_primal_trial_function_current-velocity_primal_trial_function_previous,
            velocity_primal_test_function,
        )
        * cell_integral
        + time_step
        * ufl.inner(
            ufl.grad(velocity_primal_trial_function_current),
            ufl.grad(velocity_primal_test_function),
        )
        * cell_integral
     )

$A_{2}((u,z),v)= \gamma_M\tau \sum^{N}_{n=1}(u^n,v^n)_{\omega}+\gamma_0(h \nabla u^0,h \nabla v^0) +\gamma_1 \tau\sum^{N}_{n=1}(\tau \nabla \partial_{\tau}u^n, \tau \nabla \partial_{\tau}v^n) +\tau\sum^{N}_{n=1}\sum_{F \in \mathcal{F}_h}\frac{1}{h}(\mathcal{J}(u^n),\mathcal{J}({v^n}))_F+\tau \sum^{N}_{n=1}((\partial_{\tau}v^n,z^n)+a_h(v^n,z^n))$

In [13]:
def primal_stabilizer_bilinear_form(
    velocity_primal_trial_function_current,
    velocity_primal_trial_function_previous,
    velocity_primal_test_function_current,
    velocity_primal_test_function_previous,
    velocity_dual_trial_function,
    time_step,
    mesh,
    spatial_element_degree,
    gamma_0,
    gamma_1,
    gamma_m,
):
    (
        cell_integral,
        exterior_facet_integral,
        interior_facet_integral
    ) = construct_measures(mesh, spatial_element_degree)
    cell_diameter = ufl.CellDiameter(mesh)
    return (
        #subdomain indicator term
        gamma_m
        * time_step
        #* integral_phi_phi
        * data_domain_indicator_function
        * ufl.inner(
                    velocity_primal_trial_function_current,
                    velocity_primal_test_function_current,
                )
        * cell_integral
        #  initial gradient velocity term
        # + gamma_0
        # * cell_diameter**2
        # * ufl.inner(
        #     ufl.grad(velocity_primal_trial_function_initial),
        #     ufl.grad(velocity_primal_test_function_initial),
        # )
        # * cell_integral
        # time gradient velocity term
        + gamma_1
        * time_step
        * ufl.inner(
             ufl.grad(velocity_primal_trial_function_current-velocity_primal_trial_function_previous),
            ufl.grad(velocity_primal_test_function_current-velocity_primal_test_function_previous),
        )
        * cell_integral
        # jump term
        + time_step
        * (1 / cell_diameter)
        * ufl.inner(
            ufl.jump(velocity_primal_trial_function_current),
            ufl.jump(velocity_primal_test_function_current),
        )
        * exterior_facet_integral
        #+ integral_dphi_phi
        + ufl.inner(
            velocity_primal_test_function_current-velocity_primal_test_function_previous,
            velocity_dual_trial_function,
        )
        * cell_integral
        + time_step
        #* integral_phi_phi
        * ufl.inner(
            ufl.grad(velocity_primal_test_function_current),
            ufl.grad(velocity_dual_trial_function),
        )
        * cell_integral
         
    )

In [19]:

bilinear_form = 0

linear_form = 0

for time_slice_index in range(1,number_of_time_slices+1):
    time_slice = time_slice_index * time_step
    # A_1[primal_trial, primal_test]
    bilinear_form += problem_bilinear_form(
        velocity_primal_trial_function[time_slice_index],
        velocity_primal_trial_function[time_slice_index-1],
        velocity_primal_test_function[time_slice_index],
        time_step,
        mesh,
        spatial_element_degree,
    )
    # A_2[(primal_trial, dual_trial), primal_test]
    bilinear_form += primal_stabilizer_bilinear_form(
        velocity_primal_trial_function[time_slice_index],
        velocity_primal_trial_function[time_slice_index-1],
        velocity_primal_test_function[time_slice_index],
        velocity_primal_test_function[time_slice_index-1],
        velocity_dual_trial_function[time_slice_index],
        time_step,
        mesh,
        spatial_element_degree,
        gamma_0,
        gamma_1,
        gamma_m,
    )
    # right hand side
    linear_form +=
    

IndentationError: unexpected indent (3002443929.py, line 18)

In [ ]:
if include_initial_condition:
    initial_velocity_function_space, dofmap_initial_displacement_to_mixed = (
        mixed_function_space.sub(0).sub(0).collapse()
    )
    #initial_displacement = dolfinx.fem.Function(initial_displacement_function_space)
    initial_velocity = dolfinx.fem.Function(initial_velocity_function_space)
    # initial_velocity_expression = dolfinx.fem.Expression(
    #     ufl.pi * ufl.sin(ufl.pi * spatial_coordinate[0]),
    #     initial_velocity_function_space.element.interpolation_points(),
    # )
    # initial_velocity.interpolate(initial_velocity_expression)
    initial_velocity.x.array[:] = 0.0
    # I_1(primal_trial, primal_test)
    # bilinear_form += (
    #     gamma_primal_jump
    #     * (1 / time_step)
    #     * ufl.inner(
    #         displacement_primal_trial_function[0], displacement_primal_test_function[0]
    #     )
    #     * cell_integral
    #     + gamma_primal_jump
    #     * time_step
    #     * ufl.inner(
    #         ufl.grad(displacement_primal_trial_function[0]),
    #         ufl.grad(displacement_primal_test_function[0]),
    #     )
    #     * cell_integral
    # ) #
    # I_2(primal_trial, primal_test)
    bilinear_form += (
        gamma_0
        * (1 / time_step)
        * ufl.inner(
            velocity_primal_trial_function[0], velocity_primal_test_function[0]
        )
        * cell_integral
    ) #
    # linear_form += (
    #     gamma_primal_jump
    #     * (1 / time_step)
    #     * ufl.inner(initial_displacement, displacement_primal_test_function[0])
    #     * cell_integral
    #     + gamma_primal_jump
    #     * time_step
    #     * ufl.inner(
    #         ufl.grad(initial_displacement),
    #         ufl.grad(displacement_primal_test_function[0]),
    #     )
    #     * cell_integral
    # )
    linear_form += (
        gamma_primal_jump
        * (1 / time_step)
        * ufl.inner(initial_velocity, velocity_primal_test_function[0])
        * cell_integral
    )

Assemble matrix for bilinear form (left-hand side of linear system)

Set up solver